In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
DATA_DIRECTORY = "/data/users/bdrummon/MetOfficeBirdSurvey"

SAVE_DIRECTORY = "/data/users/bdrummon/MetOfficeBirdSurvey/plots"

SKIP_FOOTER = {"2018" : 3,
               "2019" : 1,
               "2020" : 0,
               "2021" : 0,
               "2022" : 3}

YEARS = ["2018", "2019", "2020", "2021", "2022"]

SAVE = True

In [ ]:
def load_dataframe():
    # Load data and create dataframe with date as index and species names as columns
    # For records that are recorded as a minimum (e.g. 20+), assume the minimum number
    # For records that are recorded as approximate (e.g. c30), assume the estimate
    # For species marked as present only, assume 1
    # For species marked as hear, assume 1
    # For species with a range given, calculate mean (nearest integer)
    
    df_in = pd.DataFrame()
    
    for year in YEARS:
        data = pd.read_excel(f"{DATA_DIRECTORY}/MO_BWG_birdsurvey_{year}_DBRCformat_v3.xlsx",
                      usecols=[3,6,9], names = ["species", "date", "count"],
                        skipfooter=SKIP_FOOTER[year],
                         dtype={"date" : str})
        df_in = df_in.append(data)

        
    
    # Remove blank lines
    df_in = df_in.dropna()

    # Handle non integers
    df_in["count"] = df_in["count"].str.replace(" present", "")
    df_in["count"] = df_in["count"].str.replace("+", "")
    df_in["count"] = df_in["count"].str.replace("c", "")
    df_in["count"] = df_in["count"].str.replace("present", "1")
    df_in["count"] = df_in["count"].str.replace("heard", "1")
    range_values = [i for i in df_in["count"].to_list() if "-" in i]
    means = []
    for range_value in range_values:
        lower = range_value.split("-")[0]
        upper = range_value.split("-")[1]
        means.append(str(int((float(lower)+float(upper))/2.)))
    for range_value, mean in zip(range_values,means):
        df_in["count"] = df_in["count"].str.replace(range_value, mean)
    
    df_in["count"] = df_in["count"].astype(int)
    
    # Convert time column to datetime
    dates = df_in["date"].to_list()
    dates = [datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S") for i in dates]
    df_in["date"] = dates

    # Get unique list of species
    species_list = set(df_in["species"].to_list())

    # Get unique list of dates
    unique_dates_list = sorted(set(dates))

    # Make full dataframe
    df_full = pd.DataFrame(index=sorted(unique_dates_list), columns=species_list)

    for species in species_list:
        counts = []
        years = []
        for date in unique_dates_list:
            df = df_in[df_in["species"]==species]
            df = df[df["date"]==date]
            if df.empty:
                counts.append(0)
            else:
                counts.append(df["count"].to_list()[0])

        df_full[species] = np.array(counts)

    return df_full

    

In [ ]:
def plot_df(df, column, year):
    
    df[column].plot(marker='o')
    plt.title(column)
    plt.ylabel("Number of counts")
    if SAVE:
        plt.savefig(f"{SAVE_DIRECTORY}/{year}/{column}_timeseries.png", dpi=300, facecolor='white', transparent=False)
    else:
        plt.show()
    plt.clf()
    

In [ ]:
def annual_analysis(year):
    
    stats_dict = {}
    
    # Calculate statistics
    df = load_dataframe()
    
    # Add a year column
    year_list = df.index.year.to_list()
    year_list = [str(yr) for yr in year_list]
    df["year"] = year_list
    
    # Extract current year
    df = df[df["year"] == year]
    df = df.drop(columns=["year"])
    
    species_list = []
    for column in df.columns.to_list():
        if df[column].to_numpy().max() > 0:
            species_list.append(column)
            stats_dict[column] = {}
    
    
    for species in species_list:
        plot_df(df, species, year)

    # Calculate reporting rate of each species
    reporting_rate = []
    for species in species_list:
        species_array = df[species].to_numpy()
        reporting_rate.append(np.count_nonzero(species_array)/species_array.size)
        stats_dict[species]["reporting_rate"] = np.count_nonzero(species_array)/species_array.size
    # Sort in order of reporting rate
    xy = zip(reporting_rate, species_list)
    xy = sorted(xy, reverse=True)
    sorted_species_list = [y for x,y in xy]
    reporting_rate = [x for x,y in xy]

    # Calculate average and max number of individuals (when seen)
    mean_number = []
    max_number = []
    for species in species_list:
        species_array = df[species].to_numpy()
        species_array = species_array[species_array != 0]
        if species_array.size > 0:
            stats_dict[species]["mean_number"] = species_array.mean()
            stats_dict[species]["max_number"] = species_array.max()
        else:
            stats_dict[species]["mean_number"] = 0
            stats_dict[species]["max_number"] = 0

    # Calculate number of species seen on each survey date
    number_species_each_survey = []
    for row in df.iterrows():
        number_species_each_survey.append(np.count_nonzero(row[1].to_list()))
    number_species_each_survey = np.array(number_species_each_survey)
    df["number_species"] = number_species_each_survey
    # Plot result
    plot_df(df, "number_species", year)

    # Write to file 
    f = open(f"{SAVE_DIRECTORY}/{year}/summary.txt", "w")
    f.write(f"Summary of {year} BWG bird surveys\n")
    f.write(f"Number of surveys: {len(df.index)}\n")
    f.write(f"Total number of species: {len(species_list)}\n")
    f.write(f"Average number of species each survey: {round(number_species_each_survey.mean(),2)}\n")
    f.write(f"Average number of species each survey: {number_species_each_survey.max()}\n")
    f.write(f"Year list (species name, reporting rate, mean number, max number):\n")
    for i,species in enumerate(sorted_species_list):
        report_rate = round(stats_dict[species]["reporting_rate"],2)
        mean_num = round(stats_dict[species]["mean_number"],2)
        max_num = stats_dict[species]["max_number"]
        f.write(f"  {species} : {report_rate} : {mean_num} : {max_num} \n")

    f.close()
    
    return stats_dict


In [ ]:
long_term_dict = {}
for year in YEARS:
    long_term_dict[year] = annual_analysis(year)

In [ ]:
# Long term plots
for species in ["moorhen", "greenfinch", "house martin", "goldfinch", "carrion crow", "chiffchaff"]:
    report_rate = []
    mean_number = []
    for year in YEARS:
        report_rate.append(long_term_dict[year][species]["reporting_rate"])
        mean_number.append(long_term_dict[year][species]["mean_number"])
    fig,ax = plt.subplots()    
    plt.plot(YEARS, report_rate, label='Reporting rate')
    ax.set_ylabel("Reporting rate")
    
    ax2=ax.twinx()
    ax2.plot(YEARS, mean_number, label="Mean number", color='black')
    ax2.set_ylabel("Mean number")
    
    ax.legend()
    plt.title(species)
    
    if SAVE:
        plt.savefig(f"{SAVE_DIRECTORY}/{species}_long_term_timeseries.png", dpi=300, facecolor='white', transparent=False)
    else:
        plt.show()
    plt.clf()
    
    

    
    
    
    
